In [1]:
import os
import numpy as np
import glob
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
from torch.utils.data.dataset import *
import torchvision
import torch.nn.functional as F
from torchvision import transforms
from tensorboardX import SummaryWriter

In [2]:
from PIL import Image
for folder in ('data/train/hotdog', 'data/train/not_hotdog', 'data/test'):
    for f in os.listdir(folder):
        if (os.stat(os.path.join(folder, f)).st_size == 0):
            os.remove(os.path.join(folder, f))
        else:
            with Image.open(os.path.join(folder, f)) as im:
                a = np.array(im)
            if (2 == len(a.shape)): # grey
                rgb = np.empty((a.shape[0], a.shape[1], 3), dtype=a.dtype)
                for i in range(a.shape[0]):
                    for j in range(a.shape[1]):
                        rgb[i, j, 0] = rgb[i, j, 1] = rgb[i, j, 2] = a[i, j]
                Image.fromarray(rgb).save(os.path.join(folder, f))

# $k$NN

In [3]:
from itertools import chain
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

class ToPlain(object):
    def __call__(self, im):
        return np.array(im).flatten()
ds = datasets.ImageFolder('data/train', transform=transforms.Compose([transforms.Resize(50),
                                                                      transforms.CenterCrop(50),
                                                                      ToPlain()]))
x = []
y = []
for i in range(len(ds)):
    x.append(ds[i][0])
    y.append(ds[i][1])
cov = pd.DataFrame(x).cov()
n_neighbors = tuple(chain(range(1,10), range(10, 1000, 200)))
knn = RandomizedSearchCV(param_distributions =
    {'n_neighbors': n_neighbors, 'metric': ('manhattan',)},
    estimator = KNeighborsClassifier(algorithm='brute'), cv = 5, n_iter = 10) # haha
knn.fit(x, y)
print(knn.best_score_)

ones = 0
for i in y:
    if (0 == i):
        ones += 1
print(ones / len(y))

0.7162818955042527
0.43013365735115433


Мне казалось, что $k$NN будет бесполезен, но он сумел что-то выучить. Точность больше априорной вероятности.

# SVM with VGG

In [4]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
original_model = models.vgg16(pretrained=True)

featureExtractor = nn.Sequential(*list(original_model.children())[:-1])

data_transforms = transforms.Compose([
        transforms.Resize(32),
        transforms.CenterCrop(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])

data_dir = 'data'
image_datasets = {'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=data_transforms)}
train, test = random_split(image_datasets['train'], (int(len(image_datasets['train'])*0.8), len(image_datasets['train']) - int(len(image_datasets['train'])*0.8)))

train.dataset.transform = test.dataset.transform = data_transforms
train.dataset.transforms = test.dataset.transform = data_transforms
dataset_sizes = {'train': len(train), 'val': len(test)}

dataloaders = {'train': torch.utils.data.DataLoader(train, batch_size=64,
                                             shuffle=True, num_workers=4),
               'val': torch.utils.data.DataLoader(test, batch_size=64,
                                             shuffle=True, num_workers=4)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

x = []
y = []
with torch.set_grad_enabled(False):
    for i, im in enumerate(train):
        x.append(featureExtractor(im[0].unsqueeze(0)).numpy().flatten())
        y.append(im[1])
svm = RandomizedSearchCV(param_distributions =
    {'C': np.logspace(-2, 2, num=100), 'kernel': ('linear',)},
    estimator = SVC(), cv = 5, n_iter = 10)
svm.fit(x, y)
print(svm.best_score_)

0.8351063829787234


# Simple CNN

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(degrees=10, expand=True), # test different interpolations, test noize
        transforms.Resize(32),
        transforms.CenterCrop(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
#         transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225]) # TODO normalize
    ]),
    'test': transforms.Compose([
        transforms.Resize(32),
        transforms.CenterCrop(32),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
#         transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                  std=[0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data'
image_datasets = {'train': datasets.ImageFolder(os.path.join(data_dir, 'train'))}
train, test = random_split(image_datasets['train'], (int(len(image_datasets['train'])*0.8), len(image_datasets['train']) - int(len(image_datasets['train'])*0.8)))

train.dataset.transform = data_transforms['
                                          ']
test.dataset.transform = data_transforms['test']
print(train.dataset.transform)
print(test.dataset.transform) # hmm...
dataset_sizes = {'train': len(train), 'val': len(test)}

dataloaders = {'train': torch.utils.data.DataLoader(train, batch_size=64,
                                             shuffle=True, num_workers=2),
               'val': torch.utils.data.DataLoader(test, batch_size=64,
                                             shuffle=True, num_workers=2)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Fastest(nn.Module):
    def __init__(self):
        super(Fastest, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


fastest = Fastest()

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    writer = SummaryWriter()
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


            # Log scalar values (scalar summary)
            info = { 'epoch_loss': epoch_loss, 'epoch_acc': epoch_acc }

            for tag, value in info.items():
                writer.add_scalar(tag, value, epoch)            

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(fastest.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
device='cpu'
fastest = train_model(fastest, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=7)

Compose(
    Resize(size=32, interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(32, 32))
    RandomHorizontalFlip(p=0.5)
    ToTensor()
)
Compose(
    Resize(size=32, interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(32, 32))
    RandomHorizontalFlip(p=0.5)
    ToTensor()
)
Epoch 0/6
----------
train Loss: 2.2963 Acc: 0.2561
val Loss: 2.2572 Acc: 0.5182

Epoch 1/6
----------
train Loss: 2.1821 Acc: 0.5828
val Loss: 2.1301 Acc: 0.5182

Epoch 2/6
----------
train Loss: 2.0374 Acc: 0.5828
val Loss: 1.9620 Acc: 0.5182

Epoch 3/6
----------
train Loss: 1.7794 Acc: 0.5828
val Loss: 1.5094 Acc: 0.5182

Epoch 4/6
----------
train Loss: 1.0259 Acc: 0.5828
val Loss: 0.7171 Acc: 0.5000

Epoch 5/6
----------
train Loss: 0.7005 Acc: 0.5312
val Loss: 0.6935 Acc: 0.5727

Epoch 6/6
----------
train Loss: 0.6785 Acc: 0.5897
val Loss: 0.7593 Acc: 0.5182

Training complete in 1m 33s
Best val Acc: 0.572727


Сеть вообще ничего не выучила. Ей подавалось маленькое разрешение картики, не было нормализации ни на одном из слоев, да и сама сеть состоит из 2 сверток. Такая конфигурация проигрывает даже $k$NN.

# ResNet

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(degrees=10, expand=True), # test different interpolations, test noize
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ]),
} # abuse TenCrop when sending answer or rotation

data_dir = 'data'
image_datasets = {'train': datasets.ImageFolder(os.path.join(data_dir, 'train'))}
train, test = random_split(image_datasets['train'], (int(len(image_datasets['train'])*0.8), len(image_datasets['train']) - int(len(image_datasets['train'])*0.8)))

train.dataset.transform = data_transforms['train']
test.dataset.transform = data_transforms['test']
dataset_sizes = {'train': len(train), 'val': len(test)}

dataloaders = {'train': torch.utils.data.DataLoader(train, batch_size=64,
                                             shuffle=True, num_workers=4),
               'val': torch.utils.data.DataLoader(test, batch_size=64,
                                             shuffle=True, num_workers=4)}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    writer = SummaryWriter('logs')
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            # Log scalar values (scalar summary)
            info = { 'epoch_loss': epoch_loss, 'epoch_acc': epoch_acc }

            for tag, value in info.items():
                writer.add_scalar(tag, value, epoch)

#             # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [4]:
model_ft = models.resnet152(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.fc.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 1 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.1)

In [5]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------
train Loss: 0.2237 Acc: 0.8959
val Loss: 0.2220 Acc: 0.9242

Epoch 1/9
----------
train Loss: 0.1378 Acc: 0.9567
val Loss: 0.0508 Acc: 0.9848

Epoch 2/9
----------
train Loss: 0.0936 Acc: 0.9688
val Loss: 0.0566 Acc: 0.9788

Epoch 3/9
----------
train Loss: 0.0884 Acc: 0.9673
val Loss: 0.0584 Acc: 0.9788

Epoch 4/9
----------
train Loss: 0.0773 Acc: 0.9711
val Loss: 0.0616 Acc: 0.9758

Epoch 5/9
----------
train Loss: 0.0874 Acc: 0.9635
val Loss: 0.0556 Acc: 0.9818

Epoch 6/9
----------
train Loss: 0.0794 Acc: 0.9711
val Loss: 0.0588 Acc: 0.9758

Epoch 7/9
----------
train Loss: 0.0866 Acc: 0.9681
val Loss: 0.0494 Acc: 0.9818

Epoch 8/9
----------
train Loss: 0.0783 Acc: 0.9711
val Loss: 0.0537 Acc: 0.9818

Epoch 9/9
----------
train Loss: 0.0863 Acc: 0.9696
val Loss: 0.0562 Acc: 0.9788

Training complete in 6m 25s
Best val Acc: 0.984848


In [17]:
# save answers
from PIL import Image
class MyImageFolder(Dataset):
    def __init__(self, folder, transform):
        self.folder = folder
        self.transform = transform
        self.len = len(os.listdir(self.folder))
    def __getitem__(self, idx):
       return {'name': os.listdir(self.folder)[idx], 'tensor': self.transform(Image.open(os.path.join(self.folder, os.listdir(self.folder)[idx])))}
        
        


ds = MyImageFolder('data/test', transform=transforms.Compose([transforms.Resize(224),
                                                                transforms.CenterCrop(224),
                                                                transforms.ToTensor(),
                                                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]))
test_labels = open('test_labels.txt', 'w')
for i in range(ds.len):
    a = model_ft(ds[i]['tensor'].to(device).unsqueeze(0))
    
    ps = np.exp(np.array([a[0, 0].item(), a[0, 1].item()], dtype='float32'))
    ps /= np.sum(ps)
    if (ps[0] >  ps[1]):
        test_labels.write("%s %d\r\n" % (ds[i]['name'], 1))
    else:
        test_labels.write("%s %d\r\n" % (ds[i]['name'], 0))
    if (abs(ps[0] - ps[1]) < 0.3):
        print(ds[i]['name'], a)
            
test_labels.close()

00006.jpg tensor([[ 0.0500,  0.3650]], device='cuda:0')
00017.jpg tensor([[ 0.3694, -0.0985]], device='cuda:0')
00063.jpg tensor([[ 0.2430,  0.2854]], device='cuda:0')
00064.jpg tensor([[ 0.4260,  0.3902]], device='cuda:0')
00135.jpg tensor([[ 0.2146,  0.7791]], device='cuda:0')
00142.jpg tensor([[ 0.1652,  0.2136]], device='cuda:0')
00162.jpg tensor([[ 0.4804,  0.5383]], device='cuda:0')
00298.jpg tensor([[ 0.3639, -0.0189]], device='cuda:0')
